In [3]:
import cv2
import numpy as np
import time

# 加载 YOLO 模型
net = cv2.dnn.readNet('yolov3.cfg', 'yolov3.weights')

# 设置类别标签和颜色
classes = []
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
colors = [(0, 255, 0) for _ in range(len(classes))]

# 加载视频文件
cap = cv2.VideoCapture(r"D:\ALL\AIoT\GroupProject\YOLO\video2.mp4")  # 替换为您的视频文件路径

# 车辆计数
car_count = 0
current_car_count = 0
total_car_count = 0

# 定义时间间隔（秒）来进行车辆计数
interval = 1.0
start_time = time.time()

while True:
    # 读取视频的一帧
    ret, frame = cap.read()

    if not ret:
        break

        # 获取当前时间
    current_time = time.time()

    # 每间隔一秒进行一次目标检测和车辆计数
    if current_time - start_time >= interval:

        # 目标检测
        blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        outputs = net.forward(net.getUnconnectedOutLayersNames())

        class_ids = []
        confidences = []
        boxes = []

        # 解析输出结果
        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5 and class_id == 2:  # 仅检测车辆类别（class_id = 2）
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    width = int(detection[2] * frame.shape[1])
                    height = int(detection[3] * frame.shape[0])
                    x = int(center_x - width / 2)
                    y = int(center_y - height / 2)

                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, width, height])

        # 进行非最大抑制
        # indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        current_car_count = len(indices)

        print("Total Car Count in Current Frame:", current_car_count)

        total_car_count += current_car_count
        start_time = current_time


    # 显示车辆计数结果
    cv2.putText(frame, f"Car Count: {current_car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 显示处理后的帧
    cv2.imshow('Video', frame)

    # 按下 'q' 键退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放视频资源
cap.release()

# 关闭所有窗口
cv2.destroyAllWindows()

Total Car Count in Current Frame: 8
Total Car Count in Current Frame: 9
Total Car Count in Current Frame: 13
Total Car Count in Current Frame: 14
Total Car Count in Current Frame: 12
Total Car Count in Current Frame: 9
Total Car Count in Current Frame: 5
